- possible data
    - news articles + meta data - counts, etc
        - possibly per industry
    - volume
    - as triggered by earnings call or news release
    - possibly seeing source of volume?
- get targets
    - stock events - buy --> sell in 3 months. maybe sell in 1 year?
        - buy screening event - rapid drop over 1-7 days (plus high 'panic' amount)

In [1]:
from __future__ import division
from pandas_datareader import data as pddr
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import numpy as np

%matplotlib inline

def get_sp_tickers():
    ''' Scrapes list of S&P 500 companies and ticker symbols from Wikipedia'''
    
    WIKI_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

    req = requests.get(WIKI_URL)
    soup = BeautifulSoup(req.content, 'lxml')
    table_classes = {"class": ["sortable", "plainrowheaders"]}
    wikitables = soup.findAll("table", table_classes)

    rows = wikitables[0].findAll('tr')
    headers = [i.text for i in rows[0].findAll('th')]
    table_data = map(lambda x:[i.text for i in x.findAll('td')], rows[1:])
    sp = pd.DataFrame(table_data, columns = headers)
    sp['Ticker symbol'] = sp['Ticker symbol'].astype(str)
    return sp

sp_df = get_sp_tickers()

In [2]:
def batch_data_pull(tickers, start_date, end_date, batch_size = 200):
    '''Takes in a list of ticker symbols, and grabs all the Yahoo stock data between start and end dates'''
    
    assert len(tickers) > batch_size, 'Not a batch pull buddy'
    batches = int(round(len(tickers) / batch_size))
    ticker_batches = np.array_split(tickers, batches)
    raw_data = []
    data_source = 'yahoo'
    
    for ticker_batch in ticker_batches:
        # User pandas_reader.data.DataReader to load the desired data. As simple as that.
        panel_data = pddr.DataReader(ticker_batch, data_source, start_date, end_date)
        raw_data.append(panel_data)
        data = pd.concat(raw_data, axis=2)
    return data

stock_data = batch_data_pull(sp_df['Ticker symbol'], '2017-06-01', '2017-10-01')

/Users/allen/anaconda2/lib/python2.7/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: 'BRK.B', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/allen/anaconda2/lib/python2.7/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: 'BF.B', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


# Getting targets
### IDing - 5% drop in one day, net change in 1 week

In [126]:
delta = (stock_data.loc['Close'] - stock_data.loc['Close'].shift(-1)) / stock_data.loc['Close'] * 100
stack = delta.stack()
high_stack = stack[stack < -5]
high_stack = high_stack.reset_index()
after_mask = high_stack['Date'] < (stock_data['Close'].index.max() - pd.Timedelta('7 days'))
before_mask = high_stack['Date'] > (stock_data['Close'].index.min() - pd.Timedelta('7 days'))
high_stack = high_stack[after_mask & before_mask]
high_stack['level_1'].value_counts()


bad_dates = set((high_stack['Date'] + pd.Timedelta('7 days')).dt.strftime('%Y-%m-%d')) \
            - set(stock_data['Close'].index.strftime('%Y-%m-%d'))

bad_dates = pd.Series(list(bad_dates))
bad_dates = pd.to_datetime(bad_dates)
bad_dates = bad_dates - pd.Timedelta('7 days')

bd_mask = high_stack['Date'].isin(pd.to_datetime(bad_dates))
high_stack = high_stack[~bd_mask]

high_stack['date_value'] = high_stack.apply(lambda x: stock_data['Close'].loc[x['Date'], x['level_1']], axis=1)
high_stack['week_later_values'] = high_stack.apply(lambda x: stock_data['Close'].loc[x['Date'] + pd.Timedelta('7 days'), x['level_1']], axis=1)

high_stack['week_later_gain'] = (high_stack['week_later_values'] - \
                                 high_stack['date_value']) / high_stack['date_value'] * 100

high_stack.rename(columns = {0:'per_drop_from_yesterday', 'level_1':'ticker'}, inplace=True)

In [132]:
high_stack

,Date,ticker,per_drop_from_yesterday,date_value,week_later_values,week_later_gain
11,2017-09-22,DVA,-6.557655,57.490002,59.389999,3.304917
12,2017-09-20,GIS,-6.152967,52.169998,52.430000,0.498375
13,2017-09-20,LB,-5.372012,37.230000,42.759998,14.853607
14,2017-09-20,QRVO,-5.688282,70.320000,70.250000,-0.099545
15,2017-09-19,AYI,-6.450588,157.660004,165.029999,4.674613
16,2017-09-19,AZO,-5.271029,535.190002,587.130005,9.704965
17,2017-09-19,BBY,-8.699773,52.759998,54.639999,3.563308
18,2017-09-19,CNC,-5.346406,89.779999,92.320000,2.829139
19,2017-09-19,COO,-5.559556,235.990005,235.850006,-0.059324
20,2017-09-19,EVHC,-10.577585,43.110001,43.270000,0.371141


For each target:
- fundamental data
    - volume of trades over the week
        - sum stats ohh yeaahhhhh
        - maybe some detrending too bebe
        - need to normalize - compare to last weeks numbers, maybe also last months average?
    - earnings calls or not?
        - time since last earnings call
- get ALLLLLLL the news articles
    - find a way to get aforementioned news articles
    - 